In [1]:
import requests # to get the HTML
from bs4 import BeautifulSoup # to get parse the text
import numpy as np
import pandas as pd
import tqdm
from datetime import date, timedelta

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
pd.options.display.max_colwidth = 200

In [3]:
base_url = 'https://www.jobs.bg/'
url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
#      https://www.jobs.bg/front_job_search.php?frompage=15&add_sh=1&from_hp=1&term=#paging

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
def get_pages(soup):
    """Get the total number of job offerings and the total number of pages.
    output: total number of pages
    """
    total_job_number = soup.find('td',
                            attrs={'style':
                                  'height:25px;width:220px;font-weight:500;padding-bottom:5px;'}).text.split()[-1]
    total_job_number= int(total_job_number)
    total_pages = int(total_job_number/15)+1 #round up
    return total_pages

In [37]:
# total_pages = get_pages(soup)
total_pages = 30
results =[]

#for page in tqdm.tqdm(range(total_pages)):
for page in tqdm.tqdm(range(1, 2)):

    url = f'https://www.jobs.bg/front_job_search.php?frompage={page*15}&add_sh=1&from_hp=1&term=#paging'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})
    
    for i in range(0, 45, 3):
        text_info_elem = job_infos[i]
        job_logo_elem = job_infos[i+1]
        job_company_elem = job_infos[i+2]
        position_id = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href'].lstrip('job/')
        position_url = base_url+text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href']
        
        position_r = requests.get(position_url)
        position_soup = BeautifulSoup(position_r.text, 'html.parser')
        
        iframe_elem = position_soup.find('iframe', attrs = {'id':'customJobIframe'})
        if iframe_elem:
            iframe_url = iframe_elem.attrs['src']
            iframe_r = requests.get(iframe_url)
            iframe_soup = BeautifulSoup(iframe_r.text, 'html.parser')
            job_text = iframe_soup.text.strip()
        else:
            job_text = np.nan
        #Add the non iframe positions (descriptions)
        
        job_date = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'}).text.strip()
        if job_date == 'днес':
            job_date = date.today().strftime("%d.%m.%y")
        elif job_date == 'вчера':
            job_date = (date.today() - timedelta(days=1)).strftime("%d.%m.%y")

        rating_elem = text_info_elem.find('span',attrs={'class':'iconed'})

        if rating_elem:
            rataing_elem_styles = rating_elem["style"].split(";")
            rating_elem_color_styles = [x for x in rataing_elem_styles if "color:" in x]
            rating_color = rating_elem_color_styles[0].split(":")[1]
            rating_full = rating_elem.text.strip().count("\uf4b3")
            rating_half = (rating_elem.text.strip().count("\uf4b1"))/2
            rating_final = rating_full + rating_half
        else:
            rating_color = np.nan
            rating = np.nan
       
        position = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'}).text.strip()
        benefits = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'}).text.strip()
        
        tech_list = []
        for a in text_info_elem.find_all('img'):
            tech = a.attrs['alt']
            tech_list.append(tech)
        tech_list = ", ".join(tech_list)
        results.append([position_id,position_url,job_text,job_date, rating_final, rating_color, position, benefits, tech_list])

100%|██████████| 1/1 [00:03<00:00,  3.35s/it]


In [38]:
jobs_df = pd.DataFrame(results, columns=["id","url","Job Description","date", "rating", "rating_color", "position", "benefits", "Technologies"])
jobs_df

,id,url,Job Description,date,rating,rating_color,position,benefits,Technologies
0,5665209,https://www.jobs.bg/job/5665209,Technical Support Engineer\n\n\r\n Are you a natural problem solver? Do you have an interest in different technologies and IT? Do you think you can deliver exceptional customer serv...,29.04.21,5.0,#66c1ff,Technical Support Engineer,София,
1,5653591,https://www.jobs.bg/job/5653591,"Start-up feeling with security of global German client.We are developing solutions for the majority of the world’s top 30 pharmaceutical and biotech companies. From the heart of Sofia, we deliver ...",29.04.21,5.0,#66c1ff,Java developer,София; Възможност за работа от вкъщи; Дистанционно интервю,"Java, SQL"
2,5656091,https://www.jobs.bg/job/5656091,NaN,29.04.21,5.0,#66c1ff,Senior Customer Technical Support and Sales Agent (with German),София; Заплата от 1800 до 2200 BGN (Нето),"Английски, Немски"
3,5650141,https://www.jobs.bg/job/5650141,"Описание и изисквания : ""Pizza Lab"" е сред лидерите за най-бързо приготвяне на пици в България. Pizza Lab очаква своите бъдещи колеги на ул. ""Самара 4"", ж.к. Младост 4, новата сградата на Advance ...",29.04.21,5.0,#66c1ff,Асистент в кухня,София,
4,5640420,https://www.jobs.bg/job/5640420,NaN,29.04.21,5.0,#66c1ff,2nd Line IT support Technician,София; Заплата от 1800 до 2400 BGN (Нето),"Active Directory, Windows, VMware, Cisco"
5,5667932,https://www.jobs.bg/job/5667932,"Вие искате да работите в креативна и диманична среда, в която да проявите своя хъс и енергия!Вие искате ежедневно да можете да виждате резултатите от вашите действия!Вие искате да работите в мотив...",29.04.21,5.0,#66c1ff,МЕНИДЖЪР КЛИЕНТИ,София; Дистанционно интервю,
6,5646978,https://www.jobs.bg/job/5646978,„Tessi Groupe” (13 000+ members) – is a leading company in the documents and data processing business. In connection with our continuous growth we are looking for new members to join our 560+ empl...,29.04.21,5.0,#66c1ff,Customer Service Representative with fluent French – inbound calls (standard working time),София; Заплата от 2400 до 2800 BGN (Бруто),Френски
7,5644272,https://www.jobs.bg/job/5644272,"Олианс България ЕАД е част „Tessi Groupe” - европейски лидер в областта на обработката на документи и други бизнес процеси. В нашия офис в София към момента сме над 560 служители, като очакваме да...",29.04.21,5.0,#66c1ff,Оператор обработка и въвеждане данни на компютър със средно ниво Френски език,София; Заплата 1400 BGN (Бруто),Френски
8,5673623,https://www.jobs.bg/job/5673623,Are you ready to take the next step in your career and to achieve your next challenge in Sofia?If your answer is YES – join the leader of the BPO sector TELUS International Europe!Get this Opportu...,29.04.21,5.0,#66c1ff,Rent-a-car Representative with German - Sofia Summer Opportunity,София; Дистанционно интервю,"Английски, Немски"
9,5673618,https://www.jobs.bg/job/5673618,Are you ready to take the next step in your career and to achieve your next challenge in Plovdiv?If your answer is YES – join the leader of the BPO sector TELUS International Europe!Get this Oppor...,29.04.21,5.0,#66c1ff,Rent-a-car Representative with German - Plovdiv Summer Opportunity,Пловдив; Дистанционно интервю,"Английски, Немски"


In [15]:
job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})

for i in range(0, 45, 3):
    text_info_elem = job_infos[i]
    job_logo_elem = job_infos[i+1]
    job_company_elem = job_infos[i+2]
    position_id = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href'].lstrip('job/')
    position_url = base_url+text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href']
    
    position_r = requests.get(position_url)
    position_soup = BeautifulSoup(position_r.text, 'html.parser')
    iframe_url = position_soup.find('iframe', attrs = {'id':'customJobIframe'}).attrs['src']
    iframe_r = requests.get(iframe_url)
    iframe_soup = BeautifulSoup(iframe_r.text, 'html.parser')

In [19]:
position_url

'https://www.jobs.bg/job/5673606'

In [27]:
iframe_url = position_soup.find('iframe', attrs = {'id':'customJobIframe'}).attrs['src']

In [29]:
iframe_r = requests.get(iframe_url)
iframe_soup = BeautifulSoup(iframe_r.text, 'html.parser')

In [34]:
iframe_soup.text.strip()

'Astrea Recruitment\xa0was found in 2007 with one principal mission – to be different.We transformed this vision into a key driver of our efforts to be always available when our clients or partners need us to advise and guide them in taking important decisions.Our partners are increasing continuously and their satisfaction is the best business card for us.Do you like helping people and working with the team to support a customer experience? Do you like learning new skills and developing your abilities?For one of our clients – one of the world leaders in the outsourcing business – we are looking for professional, career-oriented individuals to join our client`s team for the role of Customer\xa0Service Expert with Hungarian, Slovak, German, French, Polish & English.\xa0Essential Duties and Responsibilities:* Listen and respond to customers’ needs and concerns;* Provide information about products and services;* Accurate data entry in consumer database;* Monitoring and documenting queries,